### Résolution de Sudoku avec Z3

Dans ce notebook, nous allons explorer comment utiliser Z3, un puissant SMT solver développé par Microsoft, pour résoudre des puzzles de Sudoku. Nous commencerons par une implémentation simple en utilisant des entiers, puis nous introduirons des déclinaisons plus sophistiquées.

#### 1. Introduction à Z3

Z3 est un SMT solver (Satisfiability Modulo Theories) qui peut être utilisé pour résoudre des problèmes de logique du premier ordre. Il est particulièrement efficace pour résoudre des problèmes impliquant des contraintes complexes, comme les puzzles de Sudoku.

**Références :**
- [exemples en csharp](https://github.com/Z3Prover/z3/blob/master/examples/dotnet/Program.cs)
- [Programming Z3](https://theory.stanford.edu/~nikolaj/programmingz3.html) dans le langage z3 dédié SLib.
- [Z3Py Guide Examples](https://ericpony.github.io/z3py-tutorial/guide-examples.htm) en Python
- [Online Z3 guide](https://microsoft.github.io/z3guide/) en js, SLib, Python

#### 2. Configuration de l'environnement

On installe le package z3 ainsi que celui de Plotly pour tracer des graphiques par la suite.


In [20]:
#r "nuget: Microsoft.Z3"
#r "nuget: XPlot.Plotly.Interactive"

Installed Packages Microsoft.Z3, 4.12.2 XPlot.Plotly.Interactive, 4.1.0

## Importation des Classes de Base

Nous allons importer les classes de base définies dans le notebook précédent, fournissant notamment la représentation, le chargement et l'affichage de Sudokus, et l'infrastructure de résolution.


In [21]:
#!import Sudoku-0-Environment.ipynb

# Notebook 0: Classes de Base pour la Résolution de Sudoku

Ce notebook contient les classes de base nécessaires pour la manipulation et la résolution des grilles de Sudoku. Il sera importé dans les autres notebooks pour réutiliser ces classes.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.


## Définition de la classe SudokuGrid

Nous définissons ici la classe SudokuGrid qui représente une grille de Sudoku et fournit des méthodes pour manipuler et afficher les grilles.


## Définition de l'interface ISudokuSolver

Nous définissons ici l'interface ISudokuSolver qui sera implémentée par les différentes stratégies de résolution de Sudoku.


## Définition de la classe SudokuHelper

Nous ajoutons ici la classe SudokuHelper qui contient des méthodes utilitaires pour charger et afficher des grilles de Sudoku.


## Affichage des Puzzles de chaque Difficulté

Nous allons charger et afficher un puzzle de chaque niveau de difficulté : Facile, Moyen et Difficile.


Puzzle Facile:
-------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Puzzle Moyen:
-------------------------------
| 8  5    |       2 | 4       | 
| 7  2    |         |       9 | 
|       4 |         |         | 
-------------------------------
|         | 1     7 |       2 | 
| 3     5 |         | 9       | 
|    4    |         |         | 
-------------------------------
|         |    8    |    7    | 
|    1  7 |         |         | 
|         |    3  6 |    4    | 
-------------------------------

Puzzle Difficile:
-------------------------------
| 4       |         | 8     5 | 
|    3    |         |         | 
|         | 7       |         | 
-------------------------------
|    2    |         |    6    | 
|         |    8    | 4       | 
|         |    1    |         | 
-------------------------------
|         | 6     3 |    7    | 
| 5       | 2       |         | 
| 1     4 |         |         | 
-------------------------------

#### 3. Implémentation de base avec des entiers

Comme nous allons tester plusieurs stratégies de résolution, nous commencerons par implémenter un solver simple utilisant des entiers pour représenter les cellules du Sudoku et fournissant les méthodes pour construire les contraintes.


In [22]:
using System.Diagnostics;
using System;
using System.Collections;
using System.Collections.Generic;
using Microsoft.Z3;

public class Z3IntSolverSimple : ISudokuSolver
{
    public static Context ctx = new Context();
    public static BoolExpr _GenericContraints;
    public static IntExpr[][] CellVariables = new IntExpr[9][];
    public Z3IntSolverSimple()
    {
        for (uint i = 0; i < 9; i++)
        {
            CellVariables[i] = new IntExpr[9];
            for (uint j = 0; j < 9; j++)
                CellVariables[i][j] = (IntExpr)ctx.MkConst(ctx.MkSymbol("x_" + (i + 1) + "_" + (j + 1)), ctx.IntSort);
        }
    }
    public static BoolExpr GenericContraints
    {
        get
        {
            if (_GenericContraints == null)
            {
                _GenericContraints = GetGenericConstraints();
            }
            return _GenericContraints;
        }
    }
    
    public static BoolExpr GetGenericConstraints()
    {
        // each cell contains a value in {1, ..., 9}
        Expr[][] cells_c = new Expr[9][];
        for (uint i = 0; i < 9; i++)
        {
            cells_c[i] = new BoolExpr[9];
            for (uint j = 0; j < 9; j++)
                cells_c[i][j] = ctx.MkAnd(ctx.MkLe(ctx.MkInt(1), CellVariables[i][j]),
                                          ctx.MkLe(CellVariables[i][j], ctx.MkInt(9)));
        }
        // each row contains a digit at most once
        BoolExpr[] rows_c = new BoolExpr[9];
        for (uint i = 0; i < 9; i++)
            rows_c[i] = ctx.MkDistinct(CellVariables[i]);
        // each column contains a digit at most once
        BoolExpr[] cols_c = new BoolExpr[9];
        for (uint j = 0; j < 9; j++)
        {
            IntExpr[] column = new IntExpr[9];
            for (uint i = 0; i < 9; i++)
                column[i] = CellVariables[i][j];
            cols_c[j] = ctx.MkDistinct(column);
        }
        // each 3x3 square contains a digit at most once
        BoolExpr[][] sq_c = new BoolExpr[3][];
        for (uint i0 = 0; i0 < 3; i0++)
        {
            sq_c[i0] = new BoolExpr[3];
            for (uint j0 = 0; j0 < 3; j0++)
            {
                IntExpr[] square = new IntExpr[9];
                for (uint i = 0; i < 3; i++)
                    for (uint j = 0; j < 3; j++)
                        square[3 * i + j] = CellVariables[3 * i0 + i][3 * j0 + j];
                sq_c[i0][j0] = ctx.MkDistinct(square);
            }
        }
        BoolExpr sudoku_c = ctx.MkTrue();
        foreach (BoolExpr[] t in cells_c)
            sudoku_c = ctx.MkAnd(ctx.MkAnd(t), sudoku_c);
        sudoku_c = ctx.MkAnd(ctx.MkAnd(rows_c), sudoku_c);
        sudoku_c = ctx.MkAnd(ctx.MkAnd(cols_c), sudoku_c);
        foreach (BoolExpr[] t in sq_c)
            sudoku_c = ctx.MkAnd(ctx.MkAnd(t), sudoku_c);
        return sudoku_c;
    }
    public BoolExpr GetPuzzleConstraints(SudokuGrid grid)
    {
        BoolExpr instance_c = ctx.MkTrue();
        for (uint i = 0; i < 9; i++)
            for (uint j = 0; j < 9; j++)
                if (grid.Cells[i,j] != 0)
                {
                    instance_c = ctx.MkAnd(instance_c,
                        (BoolExpr)ctx.MkEq(CellVariables[i][j], ctx.MkInt(grid.Cells[i,j])));
                }
        return instance_c;
    }

    public SudokuGrid Solve(SudokuGrid s)
    {
        SudokuGrid solution = new SudokuGrid();
        SudokuSolve(s, ref solution);
        return solution;
    }    public void SudokuSolve(SudokuGrid grid, ref SudokuGrid solution)
    {
        var sudoku_c = GenericContraints;
        var instance_c = GetPuzzleConstraints(grid);
        Solver s = ctx.MkSolver();
        s.Assert(sudoku_c);
        s.Assert(instance_c);        
        if (s.Check() == Status.SATISFIABLE)
        {
            Model m = s.Model;
            for (uint i = 0; i < 9; i++)
            {
                for (uint j = 0; j < 9; j++)
                {
                    solution.Cells[i,j] = ((IntNum)m.Evaluate(CellVariables[i][j])).Int;
                }
            }
        }
        else
        {
            Console.WriteLine("Failed to solve sudoku");
            throw new Exception("Failed to solve sudoku");
        }
    }
}

### 7. Utilisation de vecteurs de bits

Nous allons maintenant explorer une autre piste d'amélioration en utilisant des vecteurs de bits pour représenter les cellules du Sudoku.

Comme nous testerons plusieurs types de résolution, nous introduisons une classe de base qui fournit les contraintes.

In [23]:
using Microsoft.Z3;

public abstract class Z3BitVectorSolverBase : ISudokuSolver
{
    public static Context ctx = new Context();
    public static BoolExpr _GenericContraints;
    public static BitVecExpr[][] CellVariables = new BitVecExpr[9][];

    private static Sort BitVectorSort = ctx.MkBitVecSort(4);

    public static Solver _ReusableSolver;

    public Z3BitVectorSolverBase()
    {
        for (uint i = 0; i < 9; i++)
        {
            CellVariables[i] = new BitVecExpr[9];
            for (uint j = 0; j < 9; j++)
                CellVariables[i][j] = (BitVecExpr)ctx.MkConst(ctx.MkSymbol("x_" + (i + 1) + "_" + (j + 1)), BitVectorSort);
        }
    }

    public static BoolExpr GenericContraints
    {
        get
        {
            if (_GenericContraints == null)
            {
                _GenericContraints = GetGenericConstraints();
            }
            return _GenericContraints;
        }
    }

    public static Solver ReusableSolver
    {
        get
        {
            if (_ReusableSolver == null)
            {
                _ReusableSolver = MakeReusableSolver();
            }
            return _ReusableSolver;
        }
    }

    public static Solver MakeReusableSolver()
    {
        Solver s = ctx.MkSolver();
        s.Assert(GenericContraints);
        return s;
    }

    protected static BitVecExpr GetConstExpr(int value)
    {
        return (BitVecExpr)ctx.MkNumeral(value, BitVectorSort);
    }

    public static BoolExpr GetGenericConstraints()
    {
        // each cell contains a value in {1, ..., 9}
        Expr[][] cells_c = new Expr[9][];
        for (uint i = 0; i < 9; i++)
        {
            cells_c[i] = new BoolExpr[9];
            for (uint j = 0; j < 9; j++)
                cells_c[i][j] = ctx.MkAnd(ctx.MkBVULE(GetConstExpr(1), CellVariables[i][j]),
                    ctx.MkBVULE(CellVariables[i][j], GetConstExpr(9)));
        }

        // each row contains a digit at most once
        BoolExpr[] rows_c = new BoolExpr[9];
        for (uint i = 0; i < 9; i++)
            rows_c[i] = ctx.MkDistinct(CellVariables[i]);

        // each column contains a digit at most once
        BoolExpr[] cols_c = new BoolExpr[9];
        for (uint j = 0; j < 9; j++)
        {
            BitVecExpr[] column = new BitVecExpr[9];
            for (uint i = 0; i < 9; i++)
                column[i] = CellVariables[i][j];

            cols_c[j] = ctx.MkDistinct(column);
        }

        // each 3x3 square contains a digit at most once
        BoolExpr[][] sq_c = new BoolExpr[3][];
        for (uint i0 = 0; i0 < 3; i0++)
        {
            sq_c[i0] = new BoolExpr[3];
            for (uint j0 = 0; j0 < 3; j0++)
            {
                BitVecExpr[] square = new BitVecExpr[9];
                for (uint i = 0; i < 3; i++)
                for (uint j = 0; j < 3; j++)
                    square[3 * i + j] = CellVariables[3 * i0 + i][3 * j0 + j];
                sq_c[i0][j0] = ctx.MkDistinct(square);
            }
        }

        BoolExpr sudoku_c = ctx.MkTrue();
        foreach (BoolExpr[] t in cells_c)
            sudoku_c = ctx.MkAnd(ctx.MkAnd(t), sudoku_c);
        sudoku_c = ctx.MkAnd(ctx.MkAnd(rows_c), sudoku_c);
        sudoku_c = ctx.MkAnd(ctx.MkAnd(cols_c), sudoku_c);
        foreach (BoolExpr[] t in sq_c)
            sudoku_c = ctx.MkAnd(ctx.MkAnd(t), sudoku_c);

        return sudoku_c;
    }

    public BoolExpr GetPuzzleConstraints(SudokuGrid grid)
    {
        BoolExpr instance_c = ctx.MkTrue();
        for (uint i = 0; i < 9; i++)
        for (uint j = 0; j < 9; j++)
            if (grid.Cells[i,j] != 0)
            {
                instance_c = ctx.MkAnd(instance_c,
                    (BoolExpr)ctx.MkEq(CellVariables[i][j], GetConstExpr(grid.Cells[i,j])));
            }

        return instance_c;
    }

    public abstract SudokuGrid Solve(SudokuGrid s);
}


### 4. Implémentation simple avec vecteurs de bits

Nous allons implémenter un solver simple en utilisant des vecteurs de bits.

In [24]:
using Microsoft.Z3;

public class Z3BitVectorSolverSimple : Z3BitVectorSolverBase
{
    public override SudokuGrid Solve(SudokuGrid s)
    {
        SudokuGrid solution = new SudokuGrid();
        SudokuSolve(s, ref solution);
        return solution;
    }

    public void SudokuSolve(SudokuGrid grid, ref SudokuGrid solution)
    {
        var sudoku_c = GenericContraints;
        var instance_c = GetPuzzleConstraints(grid);
        Solver s = ctx.MkSolver();
        s.Assert(sudoku_c);
        s.Assert(instance_c);

        if (s.Check() == Status.SATISFIABLE)
        {
            Model m = s.Model;
            for (uint i = 0; i < 9; i++)
            {
                for (uint j = 0; j < 9; j++)
                {
                    solution.Cells[i,j] = ((BitVecNum)m.Evaluate(CellVariables[i][j])).Int;
                }
            }
        }
        else
        {
            Console.WriteLine("Failed to solve sudoku");
            throw new Exception("Failed to solve sudoku");
        }
    }
}


### 5. Utilisation de l'API de substitution avec vecteurs de bits

Pour une meilleure performance, nous pouvons également utiliser l'API de substitution qui permet sur les bases d'un modèle générique réutilisable de remplacer des variables par des constantes, comme celles du masque à résoudre.


In [25]:
using Microsoft.Z3;


public class Z3BitVectorSolverSubstitution : Z3BitVectorSolverBase
{
    public override SudokuGrid Solve(SudokuGrid s)
    {
        SudokuGrid solution = new SudokuGrid();
        SudokuSolve(s, ref solution);
        return solution;
    }

    public void SudokuSolve(SudokuGrid grid, ref SudokuGrid solution)
    {
        var substExprs = new List<Expr>();
        var substVals = new List<Expr>();

        for (int i = 0; i < 9; i++)
        for (int j = 0; j < 9; j++)
            if (grid.Cells[i,j] != 0)
            {
                substExprs.Add(CellVariables[i][j]);
                substVals.Add(GetConstExpr(grid.Cells[i,j]));
            }

        BoolExpr instance_c = (BoolExpr)GenericContraints.Substitute(substExprs.ToArray(), substVals.ToArray());
        Solver solver = ctx.MkSolver();
        solver.Assert(instance_c);
        if (solver.Check() == Status.SATISFIABLE)
        {
            Model m = solver.Model;
            for (uint i = 0; i < 9; i++)
            {
                for (uint j = 0; j < 9; j++)
                {
                    if (grid.Cells[i,j] == 0)
                    {
                        solution.Cells[i,j] = ((BitVecNum)m.Evaluate(CellVariables[i][j])).Int;
                    }
                    else
                    {
                        solution.Cells[i,j] = grid.Cells[i,j];
                    }
                }
            }
        }
        else
        {
            Console.WriteLine("Failed to solve sudoku");
            throw new Exception("Failed to solve sudoku");
        }
    }
}


### 6. Utilisation de l'API de tactiques avec vecteurs de bits

Enfin, nous allons explorer l'utilisation de l'API de tactiques pour résoudre les Sudoku.

In [26]:
using Microsoft.Z3;
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

internal class Z3BitSub_Tactic : Z3BitVectorSolverBase
{
    public override SudokuGrid Solve(SudokuGrid s)
    {
        SudokuGrid solution = new SudokuGrid();
        SudokuSolve(s, ref solution);
        return solution;
    }
    public void SudokuSolve(SudokuGrid grid, ref SudokuGrid solution)
    {
        var substExprs = new List<Expr>();
        var substVals = new List<Expr>();
        for (int i = 0; i < 9; i++)
            for (int j = 0; j < 9; j++)
                if (grid.Cells[i,j] != 0)
                {
                    substExprs.Add(CellVariables[i][j]);
                    substVals.Add(GetConstExpr(grid.Cells[i,j]));
                }
        BoolExpr instance_c = (BoolExpr)GenericContraints.Substitute(substExprs.ToArray(), substVals.ToArray());
        Solver solver = ctx.MkSolver();
        solver.Assert(instance_c);
        BoolExpr puzzleConstraints = GetPuzzleConstraints(grid);
        Tactic tactic = ctx.MkTactic("simplify");
        Goal goal = ctx.MkGoal();
        goal.Assert(ctx.MkAnd(GenericContraints, puzzleConstraints));
        ApplyResult applyResult = tactic.Apply(goal);
        if (applyResult.NumSubgoals > 0)
        {
            Goal newGoal = applyResult.Subgoals[0];
            solver.Assert(newGoal.Formulas);
            if (solver.Check() == Status.SATISFIABLE)
            {
                Model m = solver.Model;
                for (uint i = 0; i < 9; i++)
                {
                    for (uint j = 0; j < 9; j++)
                    {
                        if (grid.Cells[i,j] == 0)
                        {
                            solution.Cells[i,j] = ((BitVecNum)m.Evaluate(CellVariables[i][j])).Int;
                        }
                        else
                        {
                            solution.Cells[i,j] = grid.Cells[i,j];
                        }
                    }
                }
            }
            else
            {
                Console.WriteLine("Failed to solve sudoku");
                throw new Exception("Failed to solve sudoku");
            }
        }
    }
}


### 7. Comparaison des solveurs

Pour comparer les différentes implémentations de solveurs, nous utiliserons une approche similaire à celle de notre notebook OR-Tools.

In [27]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.Linq;
using System.Threading;
using System.Threading.Tasks;
using XPlot.Plotly;
using Trace = XPlot.Plotly.Trace;
using Microsoft.DotNet.Interactive;

var solvers = new List<(string Name, ISudokuSolver Solver)>
{
    ("Z3 Int Solver Simple", new Z3IntSolverSimple()),
    ("Z3 Bit Vector Solver Simple", new Z3BitVectorSolverSimple()),
    ("Z3 Bit Vector Solver Substitution", new Z3BitVectorSolverSubstitution()),
    ("Z3 Bit Vector Solver Tactic", new Z3BitSub_Tactic())
};

List<(string SolverName, string Difficulty, double Time, int SolvedCount, string Status)> results = new();

var displayPlaceholder = display("Running tests...");

void TestSolver(ISudokuSolver solver, SudokuDifficulty difficulty, string solverName, int numberOfSudokus, int timeLimitMilliseconds)
{
    var sudokus = SudokuHelper.GetSudokus(difficulty).Take(numberOfSudokus).ToList();
    Stopwatch stopwatch = new Stopwatch();
    int solvedCount = 0;
    string status = "Success";

    var message = $"Testing {solverName} on {difficulty} sudokus...";

    displayPlaceholder.Update(message);

    foreach (var sudoku in sudokus)
    {
        var cts = new CancellationTokenSource();
        cts.CancelAfter(timeLimitMilliseconds);

        Task task = Task.Run(() =>
        {
            try
            {
                SudokuGrid solved = solver.Solve(sudoku);
                if (solved.NbErrors(sudoku) == 0)
                {
                    Interlocked.Increment(ref solvedCount);
                }
            }
            catch (Exception)
            {
                // Ignore exceptions for unsolvable sudokus
            }
        }, cts.Token);

        stopwatch.Start();
        if (!task.Wait(timeLimitMilliseconds))
        {
            status = "Timeout";
            break;
        }
        stopwatch.Stop();

        if (cts.Token.IsCancellationRequested)
        {
            status = "Timeout";
            break;
        }
    }

    if (solvedCount < numberOfSudokus)
    {
        status = "Disqualified";
    }

    double totalTime = stopwatch.Elapsed.TotalMilliseconds;
    results.Add((solverName, difficulty.ToString(), totalTime, solvedCount, status));
}

int numberOfSudokus = 10;
var difficulties = new[] { SudokuDifficulty.Easy, SudokuDifficulty.Medium, SudokuDifficulty.Hard };

int timeLimitMilliseconds = 3000;

foreach (var (solverName, solver) in solvers)
{
    foreach (var difficulty in difficulties)
    {
        TestSolver(solver, difficulty, solverName, numberOfSudokus, timeLimitMilliseconds);
    }
}

// Affichage des résultats pour diagnostiquer les solveurs mal-adaptés
foreach (var result in results)
{
    Console.WriteLine($"{result.SolverName} | Difficulty: {result.Difficulty} | Time: {result.Time} ms | Solved: {result.SolvedCount} | Status: {result.Status}");
}

// Préparer les données pour les graphiques Plotly 2D
var solverNames = results.Where(r => r.Status != "Disqualified").Select(r => r.SolverName).Distinct().ToArray();

foreach (var difficulty in difficulties)
{
    var difficultyResults = results.Where(r => r.Difficulty == difficulty.ToString() && r.Status != "Disqualified").ToList();
    var trace = new List<Trace>
    {
        new Bar
        {
            x = difficultyResults.Select(r => r.SolverName).ToArray(),
            y = difficultyResults.Select(r => r.Time).ToArray(),
            name = difficulty.ToString()
        }
    };

    var layout = new Layout.Layout
    {
        title = $"Comparison of Solver Performance - {difficulty} Difficulty",
        xaxis = new Xaxis { title = "Solver" },
        yaxis = new Yaxis { title = "Total Time (ms)", type = "log" }
    };

    var chart = Chart.Plot(trace);
    chart.WithLayout(layout);
    display(chart);
}


Testing Z3 Bit Vector Solver Tactic on Hard sudokus...

Z3 Int Solver Simple | Difficulty: Easy | Time: 637,2513 ms | Solved: 10 | Status: Success
Z3 Int Solver Simple | Difficulty: Medium | Time: 1475,8216 ms | Solved: 10 | Status: Success
Z3 Int Solver Simple | Difficulty: Hard | Time: 4289,8176 ms | Solved: 10 | Status: Success
Z3 Bit Vector Solver Simple | Difficulty: Easy | Time: 479,2351 ms | Solved: 10 | Status: Success
Z3 Bit Vector Solver Simple | Difficulty: Medium | Time: 627,4598 ms | Solved: 10 | Status: Success
Z3 Bit Vector Solver Simple | Difficulty: Hard | Time: 786,5088 ms | Solved: 10 | Status: Success
Z3 Bit Vector Solver Substitution | Difficulty: Easy | Time: 464,005 ms | Solved: 10 | Status: Success
Z3 Bit Vector Solver Substitution | Difficulty: Medium | Time: 575,7998 ms | Solved: 10 | Status: Success
Z3 Bit Vector Solver Substitution | Difficulty: Hard | Time: 864,8419 ms | Solved: 10 | Status: Success
Z3 Bit Vector Solver Tactic | Difficulty: Easy | Time: 495,28 ms | Solved: 10 | Status: Success
Z3 Bit Vector Solv

<!DOCTYPE html>

<!DOCTYPE html>

<!DOCTYPE html>

### Conclusion

Dans ce notebook, nous avons exploré différentes approches pour résoudre des puzzles de Sudoku en utilisant Z3. Nous avons commencé par une implémentation simple en utilisant des entiers, puis nous avons introduit des méthodes plus sophistiquées utilisant des vecteurs de bits et l'API de substitution. Nous avons également comparé les performances de ces différentes approches. Les solveurs simples sont efficaces pour des puzzles faciles, tandis que les méthodes plus sophistiquées peuvent offrir de meilleures performances pour des puzzles plus complexes. La possibilité de paramétrer les solveurs permet de choisir la méthode la plus adaptée à chaque situation.